In [ ]:
import datetime

In [ ]:
import pandas as pd
import pickle
from collections import Counter
import glob

In [ ]:
dd=pd.read_stata('Sentencing/sentencing_since_1991.dta')

In [ ]:
def combinations(line):
    l = line.strip().split(',')
    combs=[]        
    for i in range(2, len(l)+1):
        els = [",".join(x) for x in itertools.combinations(l, i)]
        combs.extend(els)
    return combs

In [ ]:
newdd=dd[['dateofsentencing','district','lengthofprisonsentence']]

In [ ]:
newdd=newdd[newdd.dateofsentencing.isnull()==False]

In [ ]:
newdd.dateofsentencing[newdd.dateofsentencing.astype(int)>500000]+=19000000

In [ ]:
newdd.dateofsentencing[newdd.dateofsentencing.astype(int)<=500000]+=20000000

In [ ]:
newdd['Date']=newdd.dateofsentencing.astype(int).astype(str)

In [ ]:
newdd['date']=pd.to_datetime(newdd['Date'])

In [ ]:
def convertlength(df):
    orig_leng = df["lengthofprisonsentence"]
    if len(orig_leng) < 2:
        return None
    elif orig_leng[-1] == "Y":
        return int(orig_leng[:-1])*12
    elif orig_leng[-1] == "M":
        return int(orig_leng[:-1])
    else:
        return None
newdd["lengthofprisonsentence"] = newdd.apply(convertlength, axis=1)

In [ ]:
newdd=newdd[newdd.lengthofprisonsentence.isnull()==False]
newdd[['district','lengthofprisonsentence','id','date']]

In [ ]:
l=dd['Length of Prison Sentence'][dd['Length of Prison Sentence'].str.endswith('M')==True]

s=l.astype(str).str[0:3]

s=l.astype(str).str[0:3]

s_r=s.reset_index()
s_r=s_r['Length of Prison Sentence']

In [ ]:
meta=pickle.load(open("metadata.pkl","rb"))

In [ ]:
group=meta.groupby('caseid', as_index=False)['judgeidentificationnumber'].agg({'list':(lambda x: list(x))})

In [ ]:
merge1=pd.merge(meta, group, how='inner', left_on='caseid', right_on='caseid')

In [ ]:
merge1.rename(columns={'judgeidentificationnumber': 'judgeid1'}, inplace=True)

In [ ]:
merge1['judgeid1']=[item[0] for item in merge1.list]
merge1['judgeid2']=[item[1] for item in merge1.list]
merge1['judgeid3']=[item[2] for item in merge1.list]

In [ ]:
merge1=merge1.drop_duplicates('caseid')

In [ ]:
merge1.drop(['JudgesListTouse','geniss1','songername','jOrigname','judgelastname','list','yearq'],axis=1)

In [ ]:
meta=merge1

In [ ]:
cc=pickle.load( open( "Sentencing/sentence_data.csv", "rb" ) )

In [ ]:
bio=pd.read_csv('Bio/JudgeBio.csv')

In [ ]:
merge1=pd.merge(cc, meta, how='inner', left_on='caseid', right_on='caseid')

In [ ]:
merge2=pd.merge(merge1, bio, how='inner', left_on='judgeid1', right_on='id')

In [ ]:
merge3=pd.merge(merge2, bio, how='inner', left_on='judgeid2', right_on='id')

In [ ]:
merge4=pd.merge(merge3, bio, how='inner', left_on='judgeid3', right_on='id')

In [ ]:
merge4.to_pickle('merged_metadata_0326.pkl')

In [ ]:
cc=merge4

cc['month_3_b']=cc.date+pd.DateOffset(months=-3)

def find_date(df):
    min_date = df["month_3_b"]
    date=df["date"]
    max_date = df["month_3"]
    a=dc[(dc['date'] >= date) & (dc['date'] <= max_date)].lengthofprisonsentence.mean()
    b=dc[(dc['date'] >= min_date) & (dc['date'] <= date)].lengthofprisonsentence.mean()

    return a-b

cc["length_3m_dif"] = cc.apply(find_date, axis=1)

In [ ]:
s=cc.dropna(subset = ['length_3m_dif'])

In [ ]:
s['txt']=None

In [ ]:
import nltk
from nltk.util import ngrams
from io import open
import unicodedata
import string
import re

In [ ]:
s.to_pickle('merged_metadata_dict.pkl')

In [ ]:
s['txt']=None

In [ ]:
for i,(year,ids,j) in enumerate(zip(s.year,s.caseid,s.judge_name)):
    filepath = "Sentencing/sentences_new/sent_"+str(year)+"/"+ids+"_*_"+j+"*.txt"
    txt = glob.glob(filepath)
    with open (txt[0],encoding="utf8") as f:
        #c=Counter(list(f.read()))
        text=f.read().lower()
        text = re.sub("\n", ' ', text)
        s.txt[i]=text

In [ ]:
s.to_pickle('training_data.pkl')